# Lab Template

## import libraries

In [ ]:
import asyncio
import math
from math import log, sin, cos, tan, exp, sqrt, pi
import time
from random import randrange
import torch
import numpy as np
from classroom import Classroom
from classroom import Student
from classroom import BytesDataset
from classroom import GutenbergSnippetsDataset
from classroom import MLPLM, MyLM
from classroom import TransformerLM
from classroom import AdamW
from classroom import Sonny
from classroom import Floyd
from classroom import Plot
from classroom import Fun
from classroom import Count
from classroom import Sum
from classroom import Diff
from classroom import Log2Sum
from classroom import KalmanFilter1D
from classroom import MedianFilter
from classroom import TwoWindowFilter
from classroom import numel
from classroom import utf8decode

## initialize model

In [ ]:
if True:
    path = 'checkpoint.pt'
    model = torch.load(path).to('cuda')

In [ ]:
if True:
    model = (
        MyLM(
            n_ctx=256,
            n_vocab_in=256,
            d_model=8,
            n_layers=2,
            d_hidden=4096,
            nonlinearity="GELU",
            p_dropout=0.0,
            n_vocab_out=256).to('cuda'))

In [ ]:
numel(model), numel(model)*4/1E9

## initialize student

In [ ]:
optimizer = AdamW(parameters=model.named_parameters())
dataset = GutenbergSnippetsDataset()
batch_size = None
example_length = model.n_ctx + 1
student= Student(
    model=model,
    optimizer=optimizer,
    dataset=dataset,
    batch_size=batch_size,
    example_length=example_length)

## schedule hyperparameters

In [ ]:
student.batch_size=8192
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    batch_multiplier = 16
    lr_base = 1e-11
    lr = lambda n: lr_base * n #* cos(3.14159*n/batch_multiplier/256)**2
    student.optimizer.state[pn]["lr"]           = lambda n: lr(n)
    student.optimizer.state[pn]["beta1"]        = lambda n: 0.9
    student.optimizer.state[pn]["beta2"]        = lambda n: 0.999
    student.optimizer.state[pn]["weight_decay"] = lambda n: 0.001
    student.optimizer.state[pn]["update"]       = lambda n: n%batch_multiplier == 0

## initialize baseline

In [ ]:
student.reset_baseline()

## start training

In [ ]:
classroom = Classroom()

In [ ]:
classroom.enroll(student)

## autocomplete

In [ ]:
def autocomplete(prompt=None):
    for (idx, student) in enumerate(classroom.students):
        print(f"\n\nStudent #{idx}\n==========")
        print(student.autocomplete(prompt=prompt, n_generate=1024, temp=1.0))
autocomplete()

## plots

In [ ]:
import time
plot_data = {}
lag = 1024
for (idx, student) in enumerate(classroom.students):
    X = Fun(Log2Sum(), student.times)
    Y = Fun(TwoWindowFilter(lag=lag), student.grades)
    Z = Fun(TwoWindowFilter(lag=lag), student.baseline_grades)
    plot_data.update({f"grades-{idx}": (X, Y)})
    plot_data.update({f"baseline-{idx}": (X, Z)})
Plot(**plot_data)

In [ ]:
import time
plot_data_2 = {}
lag = 2000
for (idx, student) in enumerate(classroom.students):
    X = Fun(Count(), student.times)
    Y = Fun(lambda x, y: x - y, student.grades, student.baseline_grades)
    Y = Fun(TwoWindowFilter(lag=lag), Y.output, aux=Y)
    plot_data_2.update({f"improvement-{idx}": (X, Y)})
Plot(**plot_data_2)

## stats

In [ ]:
for (idx, student) in enumerate(classroom.students):
    print(f"\nStudent #{idx}\n==========")
    N = 25000
    n = len(student.times)-1
    time = student.time
    mean_grade = np.mean(np.array(student.grades[n-N:n]))
    mean_baseline_grade = np.mean(np.array(student.baseline_grades[n-N:n]))
    mean_improvement = mean_grade - mean_baseline_grade
    improvement_rate = mean_improvement / (time - student.time_of_last_baseline)
    message = '\n'.join([
        f"lr                    = {student.optimizer.state['language_model.module.layers.0.weight']['lr'](n)}",
        f"batch_size            = {student.batch_size}",
        f"example_length        = {student.example_length}",
        f"n                     = {n}",
        f"time                  = {int(time)}s",
        f"time_of_last_baseline = {int(student.time_of_last_baseline)}s",
        f"steps per second      = {(n/time)}",
        f"mean_baseline_grade   = {mean_baseline_grade}",
        f"mean_grade            = {mean_grade}",
        f"mean_improvement      = {mean_improvement}",
        f"improvement_rate      = {improvement_rate} per second",
        f"time to 81            = {(.81-mean_grade)/improvement_rate//360/10} hours",
        f"improvement per step  = {mean_improvement/n}",
        f"steps to 81           = {int((.81-mean_grade)/(mean_improvement/n))/batch_multiplier} steps"
    ])
    print(message)

In [ ]:
8192*.61*numel(model)/1E9

## save

In [ ]:
torch.save(student.model, f='checkpoint.pt')

In [ ]:
lyles_constant = 9115131782/14818489608 #* log(50257)/log(65536)
lyles_constant

In [ ]:
mg = 0.792844

In [ ]:
(1 - mg) / lyles_constant * log(50257)  # gpt2natscale (for kaplan paper comparison)

In [ ]:
(1 - mg) * 8  #  bits per character

In [ ]:
2**((1 - mg) * 8)  # perplexity

## parameter histograms

In [ ]:
histograms = []
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    with torch.no_grad():
        print(idx, pn, torch.sqrt(torch.var(p)).item())
        Y, X = np.histogram(p.detach().cpu().numpy(), bins=int(sqrt(torch.numel(p))), density=True)
        print(X.shape, Y.shape)
        histograms.append(Plot(**{f"hist-{idx}": (X.tolist(), Y.tolist())}))

In [ ]:
histograms[43] # 3 7 9 13 15 21 43

## batch-level grade histogram

In [ ]:
Y, X = np.histogram(student.grades[-5000:], bins=256, range=(0,1.0), density=True)
V, U = np.histogram(student.baseline_grades[-5000:], bins=256, range=(0,1.0), density=True)
Plot(**{f"grade-hist": (X, Y), "baseline": (U, V)})

In [ ]:
model.n_ctx, model.d_model, model.d_hidden, model.n_layers

## example-level grade histogram

In [ ]:
def get_graded_examples():
    result = []
    for batch_idx in range(256):
        print(f"batch_idx = {batch_idx}/256")
        x = student.dataset.batch(student.batch_size, student.example_length)
        with torch.no_grad():
            y = student.model(x)
            x = x.cpu().numpy()
            y = 1.0 - y.cpu().numpy()
            result.append(np.concatenate([x, y], axis=1))
    data = np.concatenate(result, axis=0)
    result = data.tolist()
    return result

In [ ]:
graded_examples = get_graded_examples()

In [ ]:
sum(x[-1] for x in graded_examples)/len(graded_examples)

In [ ]:
example_grades = [[] for _ in range(64)]
for example in graded_examples:
    grade = example[-1]
    for k in range(1,64):
        if int(example[-k-1]) in [ord(' '), ord('\n'), ord('\r')]:
            example_grades[k].append(grade)
            break

In [ ]:
tot = 0
for k in range(64):
    print(f"{k}, {len(example_grades[k])}, {sum(example_grades[k])/(len(example_grades[k])+1)}")
    tot += len(example_grades[k])
print(tot)

In [ ]:
85680/524231

In [ ]:
R = (0, 1)
def XYFor(k):
    es = example_grades[k]
    bins = int(sqrt(len(es)))
    Y, X = np.histogram(es, bins=bins, range=R, density=True)
    return (X, Y)
Plot(**{f"examples-hist-{k}": XYFor(k) for k in [1, 2, 3]})

In [ ]:
ord(' ')

In [ ]:
np.mean(example_grades)

In [ ]:
(1 - 0.7870894884999871)*8

In [ ]:
(1 - 0.8)*8

In [ ]:
(1 - 0.9)*8

In [ ]:
x = np.array([[1,2],[3,4]],dtype=np.uint8)

In [ ]:
np.unpackbits(x, axis=1)